# Construcción de features para el S&P 500

## Procedimientos

- Cargar los datos para los indicadores económicos
- Descargar datos:
    - Serie S&P 500
    - Series para los sectores del S&P 500
    - Series para las compañias
    - Series para las monedas
    - Series para los indices
- Calcular retornos
- Calcular variable de clase
- Calcular desfase temporal del retorno
- Calcular indicadores técnicos
- Cargar indicadores económicos
- Guardar features y variable de clase

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import itertools as it
import pickle as pk
import pandas as pd
from pandas_datareader import data
import fix_yahoo_finance

fix_yahoo_finance.pdr_override()

from datetime import datetime

from pyCBT.providers.scrappers import slickcharts, wikipedia
from pyCBT.common.files import exist
from talib import RSI, CCI, ROC, MOM, WILLR

from pyCBT.providers.oanda import account, historical

oanda_client = account.Client()

from alpha_vantage.timeseries import TimeSeries

avantage_client = TimeSeries(key="RL0YBD0NPUE3QYQ6", output_format="pandas")
col_mapping = {
            "1. open": "Open",
            "2. high": "High",
            "3. low": "Low",
            "4. close": "Close",
            "5. volume": "Volume"
        }

In [3]:
sp500_slick = slickcharts.get_sp500_metadata()
sp500_slick.set_index(keys="Symbol", inplace=True)
sp500_slick.sort_index(inplace=True)

sp500_wiki = wikipedia.get_sp500_metadata()
sp500_wiki.set_index(keys="Symbol", inplace=True)
sp500_wiki.sort_index(inplace=True)
sp500_wiki.drop(columns="Company", inplace=True)

sp500_md = pd.concat((sp500_wiki, sp500_slick), axis=1)
sp500_md.sort_values(by="Weight")

,Sector,Company,Weight
Symbol,,,
AAPL,Information Technology,Apple Inc.,1.0
MSFT,Information Technology,Microsoft Corporation,2.0
AMZN,Consumer Discretionary,Amazon.com Inc.,3.0
FB,Information Technology,Facebook Inc. Class A,4.0
BRK.B,Financials,Berkshire Hathaway Inc. Class B,5.0
JPM,Financials,JPMorgan Chase & Co.,6.0
JNJ,Health Care,Johnson & Johnson,7.0
XOM,Energy,Exxon Mobil Corporation,8.0
GOOG,Information Technology,Alphabet Inc. Class C,9.0


In [4]:
# definir fechas
from_date, to_date = datetime(1995, 1, 1), datetime.today().date()
# definir tickers y símbolos
indices_avantage = {
    "DJA": "^DJA",
    "DXY": "DX-Y.NYB",
    "NASDAQ": "^IXIC",
    "VIX": "^VIX",
    "VXN": "^VXN",
    "VXO": "^VXO"
}
indices_oanda = {
    "NYA": "US2000_USD",
    "DJI": "US30_USD",
    "FTSE": "UK100_GBP",
    "FCHI": "FR40_EUR",
    "DAX": "DE30_EUR",
    "SSE": "CN50_USD",
    "HSI": "HK33_HKD",
    "Nikkei": "JP225_USD"
}
sectors_stooq = {
    "Consumer disc.": "BI.F",
    "Consumer stap.": "BL.F",
    "Energy": "BM.F",
    "Financials": "BN.F",
    "Health care": "BS.F",
    "Industrials": "JG.F",
    "Inf. tech.": "JB.F",
    "Materials": "JA.F",
    "Utilities": "JE.F",
    "Real state": "BK.F"
}
# TODO: find other than alpha-vantage for stocks
# stocks_avantage = {
#     "**AAPL": "AAPL",
#     "*MSFT": "MSFT",
#     "AMZN": "AMZN",
#     "*GOOG": "GOOG",
#     "FB": "FB",
#     "*JPM": "JPM",
#     "***BRK.B": "BRK-B",
#     "*JNJ": "JNJ",
#     "**XOM": "XOM"
# }
stocks_stooq = {
    "AAPL": "AAPL.US",
    "MSFT": "MSFT.US",
    "AMZN": "AMZN.US",
    "FB": "FB.US",
    "JPM": "JPM.US",
    "BRK-B": "BRK-B.US",
    "JNJ": "JNJ.US",
    "GOOG": "GOOG.US",
    "XOM": "XOM.US"
}
cfds_oanda = {
    "Crude Oil": "WTICO_USD",
    "Gold": "XAU_USD"
}
currencies_oanda = {
    "USDCAD": "USD_CAD",
    "EURUSD": "EUR_USD",
    "GBPUSD": "GBP_USD",
    "USDCNY": "USD_CNY",
    "USDJPY": "USD_JPY"
}
if exist("../data/pickles/filter-sp500-datasets.p"):
    sp500, major_indices, sp500_sectors, major_stocks, major_cfds, major_currencies, major_economical = pk.load(open("../data/pickles/filter-sp500-datasets.p", "rb"))
else:
    # descargar S&P 500
    sp500 = historical.Candles(
        client=oanda_client,
        instrument="SPX500_USD",
        resolution="D",
        from_date=from_date.strftime("%Y-%m-%d"),
        to_date=to_date.strftime("%Y-%m-%d"),
        datetime_fmt="%Y-%m-%d",
        timezone="America/New_York"
    ).as_dataframe(index_name="Date")

    major_indices, sp500_sectors, major_stocks, major_cfds, major_currencies = {}, {}, {}, {}, {}
    # descargar indices
    for symbol in indices_avantage:
        major_indices[symbol], _ = avantage_client.get_daily_adjusted(indices_avantage[symbol], outputsize="full")
        major_indices[symbol].index.name = "Date"
        major_indices[symbol].rename(col_mapping, axis="columns", inplace=True)
        major_indices[symbol].sort_index(inplace=True)
        major_indices[symbol].index = pd.to_datetime(major_indices[symbol].index)
        major_indices[symbol].replace(to_replace=0, value=np.nan, inplace=True)
        major_indices[symbol].dropna(how="all", axis="index", inplace=True)
    for symbol in indices_oanda:
        major_indices[symbol] = historical.Candles(
            client=oanda_client,
            instrument=indices_oanda[symbol],
            resolution="D",
            from_date=from_date.strftime("%Y-%m-%d"),
            to_date=to_date.strftime("%Y-%m-%d"),
            datetime_fmt="%Y-%m-%d",
            timezone="America/New_York"
        ).as_dataframe(index_name="Date")
    # descargar sectores
    for symbol in sectors_stooq:
        sp500_sectors[symbol] = data.get_data_stooq(sectors_stooq[symbol]).sort_index()
    # descargar stocks
    for symbol in stocks_stooq:
        major_stocks[symbol] = data.get_data_stooq(stocks_stooq[symbol]).sort_index()
#     for symbol in stocks_avantage:
#         major_stocks[symbol], _ = avantage_client.get_daily_adjusted(indices_avantage[symbol], outputsize="full")
#         major_stocks[symbol].index.name = "Date"
#         major_stocks[symbol].rename(col_mapping, axis="columns", inplace=True)
#         major_stocks[symbol].sort_index(inplace=True)
#         major_stocks[symbol].index = pd.to_datetime(major_stocks[symbol].index)
#         major_stocks[symbol].replace(to_replace=0, value=np.nan, inplace=True)
#         major_stocks[symbol].dropna(how="all", axis="index", inplace=True)
    # descargar CFDs
    for symbol in cfds_oanda:
        major_cfds[symbol] = historical.Candles(
            client=oanda_client,
            instrument=cfds_oanda[symbol],
            resolution="D",
            from_date=from_date.strftime("%Y-%m-%d"),
            to_date=to_date.strftime("%Y-%m-%d"),
            datetime_fmt="%Y-%m-%d",
            timezone="America/New_York"
        ).as_dataframe(index_name="Date")
    # descargar monedas
    for symbol in currencies_oanda:
        major_currencies[symbol] = historical.Candles(
            client=oanda_client,
            instrument=currencies_oanda[symbol],
            resolution="D",
            from_date=from_date.strftime("%Y-%m-%d"),
            to_date=to_date.strftime("%Y-%m-%d"),
            datetime_fmt="%Y-%m-%d",
            timezone="America/New_York"
        ).as_dataframe(index_name="Date")
    # cargar datos económicos
    major_economical = pd.read_csv("../data/processed/filter-analysis/economic-indicators-short-actual.csv", index_col="Date", parse_dates=True).sort_index()
    # guardar pickle
    pk.dump((sp500, major_indices, sp500_sectors, major_stocks, major_cfds, major_currencies, major_economical), open("../data/pickles/filter-sp500-datasets.p", "wb"))

In [5]:
# indices
X_indices = pd.DataFrame(
    index=sp500.index,
    columns=pd.MultiIndex.from_tuples(
        tuples=list(it.product(["Indices"], sorted(major_indices.keys()))),
        names=["Type", "Name"]
    ),
    data=None
)
for symbol, df in major_indices.iteritems(): X_indices["Indices", symbol] = df.Close.shift()
# sectors
X_sectors = pd.DataFrame(
    index=sp500.index,
    columns=pd.MultiIndex.from_tuples(
        tuples=list(it.product(["S&P 500 sectors"], sorted(sp500_sectors.keys()))),
        names=["Type", "Name"]
    ),
    data=None
)
for symbol, df in sp500_sectors.iteritems(): X_sectors["S&P 500 sectors", symbol] = df.Close.shift()
# stocks
X_stocks = pd.DataFrame(
    index=sp500.index,
    columns=pd.MultiIndex.from_tuples(
        tuples=list(it.product(["Stocks"], sorted(major_stocks.keys()))),
        names=["Type", "Name"]
    ),
    data=None
)
for symbol, df in major_stocks.iteritems(): X_stocks["Stocks", symbol] = df.Close.shift()
# CFDs
X_cfds = pd.DataFrame(
    index=sp500.index,
    columns=pd.MultiIndex.from_tuples(
        tuples=list(it.product(["CFDs"], sorted(major_cfds.keys()))),
        names=["Type", "Name"]
    ),
    data=None
)
for symbol, df in major_cfds.iteritems(): X_cfds["CFDs", symbol] = df.Close.shift()
# currencies
X_currencies = pd.DataFrame(
    index=sp500.index,
    columns=pd.MultiIndex.from_tuples(
        tuples=list(it.product(["Currencies"], sorted(major_currencies.keys()))),
        names=["Type", "Name"]
    ),
    data=None
)
for symbol, df in major_currencies.iteritems(): X_currencies["Currencies", symbol] = df.Close.shift()
# technical indicators
X_technical = pd.DataFrame(
    index=sp500.index,
    columns=pd.MultiIndex.from_tuples(
        tuples=list(it.product(["Technical"], ["%R", "CCI", "MOM", "ROC", "RSI"])),
        names=["Type", "Name"]
    ),
    data=None
)
X_technical["Technical", "%R"] = WILLR(sp500.High.shift(), sp500.Low.shift(), sp500.Close.shift(), 5)
X_technical["Technical", "CCI"] = CCI(sp500.High.shift(), sp500.Low.shift(), sp500.Close.shift(), 5)
X_technical["Technical", "MOM"] = MOM(sp500.Close.shift(), 5)
X_technical["Technical", "ROC"] = ROC(sp500.Close.shift(), 5)
X_technical["Technical", "RSI"] = RSI(sp500.Close.shift(), 5)
# economical
major_economical.sort_index(axis="columns", inplace=True)
X_economical = pd.DataFrame(
    index=major_economical.index,
    columns=pd.MultiIndex.from_tuples(
        tuples=list(it.product(["Economical"], major_economical.columns.values)),
        names=["Type", "Name"]
    ),
    data=major_economical.values.copy()
)
major_economical = major_economical.reindex(labels=sp500.index, axis="index")
major_economical.fillna(method="ffill", inplace=True)
# master
# TODO: ordenar columnas por tipo de indicador
X_raw = pd.concat(
    objs=(
        X_indices.pct_change(),
        X_sectors.pct_change(),
        X_stocks.pct_change(),
        X_cfds.pct_change(),
        X_currencies.pct_change(),
        X_technical,
        X_economical
    ),
    axis="columns"
)
X_raw.insert(loc=0, column=("Indices", "S&P 500"), value=sp500.Close.shift().pct_change())
Y_raw = pd.DataFrame(
    index=sp500.index,
    data={
        "S&P 500 price": sp500.Close,
        "S&P 500 return": sp500.Close.pct_change(),
        "Class": (sp500.Close.pct_change() >= 0.0).astype(int),
    }
)
# filtrar NANs
X_raw.dropna(how="all", axis="index", inplace=True)
Y_raw = Y_raw.reindex(index=X_raw.index)

In [6]:
# guardar tablas crudas
X_raw.to_csv("../data/processed/sp500-ml/features-raw.csv", index=True)
Y_raw.to_csv("../data/processed/sp500-ml/target-raw.csv", index=True)

# guardar tablas de los últimos años
X_short = X_raw.copy()
X_short.fillna(method="ffill", inplace=True)
X_short.dropna(how="any", axis="index", inplace=True)
Y_short = Y_raw.reindex(index=X_short.index)
X_short.to_csv("../data/processed/sp500-ml/features.csv", index=True)
Y_short.to_csv("../data/processed/sp500-ml/target.csv", index=True)

# guardar tablas para Azure: sin NANs y con Class = UP/DOWN
X_azure = X_raw.copy()
X_azure.columns = X_azure.columns.droplevel("Type")
X_azure = pd.concat((X_azure, Y_raw.Class), axis="columns")
X_azure.fillna(method="ffill", inplace=True)
X_azure.dropna(how="any", axis="index", inplace=True)
# for j, col in enumerate(X_azure.values.T):
#     print all(np.isnan(np.array(col))), X_azure.columns[j]
X_azure.Class = X_azure.Class.replace(to_replace=[0.0, 1.0], value=["DOWN", "UP"])
X_azure.reset_index().to_csv("../data/processed/sp500-ml/features-for-mari.csv", index=False)